In [1]:
import os
from langchain_openai import OpenAIEmbeddings

In [2]:
openai_embed_model = OpenAIEmbeddings(model='text-embedding-3-small')

In [3]:
#Initialize Open Ai
from dotenv import load_dotenv

In [5]:
from langchain.document_loaders import JSONLoader
import json
from langchain.docstore.document import Document

In [21]:
os.getcwd()

'c:\\Users\\nitis\\Desktop\\Agentic_Ai_Learning\\Langraph'

In [22]:
from langchain.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from glob import glob
import os

# Step 1: Path to the Rag_data folder (adjust if needed)
pdf_folder = './Rag_data'
pdf_files = glob(os.path.join(pdf_folder, '*.pdf'))

In [23]:
pdf_files

['./Rag_data\\attention_paper.pdf',
 './Rag_data\\bert_paper.pdf',
 './Rag_data\\cnn_paper.pdf',
 './Rag_data\\resnet_paper_structured.pdf',
 './Rag_data\\transformer_paper.pdf',
 './Rag_data\\vision_transformer.pdf']

In [28]:
from langchain.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from glob import glob

def create_simple_chunks(file_path, chunk_size=3500, chunk_overlap=200):
    loader = PyMuPDFLoader(file_path)
    doc_pages = loader.load()
    splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    return splitter.split_documents(doc_pages)

# 🛠 Corrected folder path!
pdf_files = glob('./Rag_data/*.pdf')

paper_docs = []
for fp in pdf_files:
    paper_docs.extend(create_simple_chunks(file_path=fp))

print(f"✅ Loaded chunks: {len(paper_docs)}")


✅ Loaded chunks: 46


In [30]:
paper_docs

[Document(metadata={'producer': 'ReportLab PDF Library - www.reportlab.com', 'creator': '(unspecified)', 'creationdate': '2025-05-22T02:22:47+00:00', 'source': './Rag_data\\attention_paper.pdf', 'file_path': './Rag_data\\attention_paper.pdf', 'total_pages': 9, 'format': 'PDF 1.4', 'title': '(anonymous)', 'author': '(anonymous)', 'subject': '(unspecified)', 'keywords': '', 'moddate': '2025-05-22T02:22:47+00:00', 'trapped': '', 'modDate': "D:20250522022247+00'00'", 'creationDate': "D:20250522022247+00'00'", 'page': 0}, page_content='The attention mechanism allows neural networks to dynamically focus on different parts of the input\nsequence.\nIt was first introduced in the context of machine translation with the goal of improving alignment\nbetween source and target languages.\nAttention mechanisms assign varying levels of importance to different words or tokens in a sequence,\nimproving context comprehension.\nIn practice, attention uses query, key, and value matrices to calculate weigh

Embedding and Vector Storage

In [31]:
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma

# Initialize embedding model
openai_embed_model = OpenAIEmbeddings(model='text-embedding-3-small')

# Use only PDF chunks
total_docs = paper_docs

# Create and persist the vector DB
chroma_db = Chroma.from_documents(
    documents=total_docs,
    collection_name='my_db',
    embedding=openai_embed_model,
    collection_metadata={"hnsw:space": "cosine"},
    persist_directory="./my_db"
)


In [32]:
chroma_db = Chroma(persist_directory="./my_db",
collection_name='my_db',
embedding_function=openai_embed_model)

In [33]:
#Semantic Retrieval

# Create a similarity retriever from the vector store
similarity_retriever = chroma_db.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 5}
)

# Define your query
query = "What is machine learning?"

# Perform the search
top_docs = similarity_retriever.invoke(query)

# Display the results in a readable format
from IPython.display import display, Markdown

def display_docs(docs):
    for doc in docs:
        print('Metadata:', doc.metadata)
        print('Content Brief:')
        display(Markdown(doc.page_content[:1000]))
        print()

# Call the display function
display_docs(top_docs)


Metadata: {'creationdate': '2025-05-22T02:22:47+00:00', 'keywords': '', 'file_path': './Rag_data\\transformer_paper.pdf', 'source': './Rag_data\\transformer_paper.pdf', 'creator': '(unspecified)', 'producer': 'ReportLab PDF Library - www.reportlab.com', 'format': 'PDF 1.4', 'subject': '(unspecified)', 'creationDate': "D:20250522022247+00'00'", 'total_pages': 6, 'author': '(anonymous)', 'modDate': "D:20250522022247+00'00'", 'trapped': '', 'page': 0, 'title': '(anonymous)', 'moddate': '2025-05-22T02:22:47+00:00'}
Content Brief:


The Transformer architecture, introduced in 2017 by Vaswani et al., is built entirely on attention
mechanisms, removing recurrence entirely.
It consists of an encoder-decoder structure where both the encoder and decoder are made up of layers
containing multi-head self-attention and feed-forward networks.
Transformers leverage positional encoding to retain order information in sequences.
The architecture allows for better parallelization and efficiency during training compared to RNNs.
Transformers form the backbone of modern NLP models such as BERT, GPT, T5, and more.
They have also been adapted for use in image processing, audio modeling, and multi-modal tasks.
The Transformer architecture, introduced in 2017 by Vaswani et al., is built entirely on attention
mechanisms, removing recurrence entirely.
It consists of an encoder-decoder structure where both the encoder and decoder are made up of layers
containing multi-head self-attention and feed-forward networks.
Transformers leverage p


Metadata: {'page': 3, 'source': './Rag_data\\transformer_paper.pdf', 'producer': 'ReportLab PDF Library - www.reportlab.com', 'keywords': '', 'moddate': '2025-05-22T02:22:47+00:00', 'creationdate': '2025-05-22T02:22:47+00:00', 'total_pages': 6, 'subject': '(unspecified)', 'format': 'PDF 1.4', 'creator': '(unspecified)', 'trapped': '', 'creationDate': "D:20250522022247+00'00'", 'title': '(anonymous)', 'modDate': "D:20250522022247+00'00'", 'author': '(anonymous)', 'file_path': './Rag_data\\transformer_paper.pdf'}
Content Brief:


Transformers leverage positional encoding to retain order information in sequences.
The architecture allows for better parallelization and efficiency during training compared to RNNs.
Transformers form the backbone of modern NLP models such as BERT, GPT, T5, and more.
They have also been adapted for use in image processing, audio modeling, and multi-modal tasks.
The Transformer architecture, introduced in 2017 by Vaswani et al., is built entirely on attention
mechanisms, removing recurrence entirely.
It consists of an encoder-decoder structure where both the encoder and decoder are made up of layers
containing multi-head self-attention and feed-forward networks.
Transformers leverage positional encoding to retain order information in sequences.
The architecture allows for better parallelization and efficiency during training compared to RNNs.
Transformers form the backbone of modern NLP models such as BERT, GPT, T5, and more.
They have also been adapted for use in image processing, aud


Metadata: {'author': '(anonymous)', 'title': '(anonymous)', 'file_path': './Rag_data\\transformer_paper.pdf', 'creationDate': "D:20250522022247+00'00'", 'total_pages': 6, 'producer': 'ReportLab PDF Library - www.reportlab.com', 'creator': '(unspecified)', 'keywords': '', 'format': 'PDF 1.4', 'creationdate': '2025-05-22T02:22:47+00:00', 'moddate': '2025-05-22T02:22:47+00:00', 'source': './Rag_data\\transformer_paper.pdf', 'trapped': '', 'page': 5, 'subject': '(unspecified)', 'modDate': "D:20250522022247+00'00'"}
Content Brief:


The Transformer architecture, introduced in 2017 by Vaswani et al., is built entirely on attention
mechanisms, removing recurrence entirely.
It consists of an encoder-decoder structure where both the encoder and decoder are made up of layers
containing multi-head self-attention and feed-forward networks.
Transformers leverage positional encoding to retain order information in sequences.
The architecture allows for better parallelization and efficiency during training compared to RNNs.
Transformers form the backbone of modern NLP models such as BERT, GPT, T5, and more.
They have also been adapted for use in image processing, audio modeling, and multi-modal tasks.


Metadata: {'modDate': "D:20250522022247+00'00'", 'subject': '(unspecified)', 'producer': 'ReportLab PDF Library - www.reportlab.com', 'creator': '(unspecified)', 'moddate': '2025-05-22T02:22:47+00:00', 'source': './Rag_data\\attention_paper.pdf', 'keywords': '', 'total_pages': 9, 'title': '(anonymous)', 'author': '(anonymous)', 'format': 'PDF 1.4', 'creationDate': "D:20250522022247+00'00'", 'trapped': '', 'page': 2, 'file_path': './Rag_data\\attention_paper.pdf', 'creationdate': '2025-05-22T02:22:47+00:00'}
Content Brief:


This allows the model to capture relationships regardless of distance in the input.
Multi-head attention enables the model to jointly attend to information from different representation
subspaces.
The attention mechanism is a core part of architectures like BERT, GPT, and T5.
It supports language understanding, generation, translation, and more.
The attention mechanism allows neural networks to dynamically focus on different parts of the input
sequence.
It was first introduced in the context of machine translation with the goal of improving alignment
between source and target languages.
Attention mechanisms assign varying levels of importance to different words or tokens in a sequence,
improving context comprehension.
In practice, attention uses query, key, and value matrices to calculate weighted outputs.
Self-attention, introduced in the Transformer model, enables each token to attend to all other tokens in
the sequence.
This allows the model to capture relationships regardless of di


Metadata: {'moddate': '2025-05-22T02:22:47+00:00', 'keywords': '', 'creator': '(unspecified)', 'producer': 'ReportLab PDF Library - www.reportlab.com', 'creationDate': "D:20250522022247+00'00'", 'creationdate': '2025-05-22T02:22:47+00:00', 'format': 'PDF 1.4', 'source': './Rag_data\\attention_paper.pdf', 'file_path': './Rag_data\\attention_paper.pdf', 'total_pages': 9, 'subject': '(unspecified)', 'title': '(anonymous)', 'modDate': "D:20250522022247+00'00'", 'page': 3, 'trapped': '', 'author': '(anonymous)'}
Content Brief:


It supports language understanding, generation, translation, and more.
The attention mechanism allows neural networks to dynamically focus on different parts of the input
sequence.
It was first introduced in the context of machine translation with the goal of improving alignment
between source and target languages.
Attention mechanisms assign varying levels of importance to different words or tokens in a sequence,
improving context comprehension.
In practice, attention uses query, key, and value matrices to calculate weighted outputs.
Self-attention, introduced in the Transformer model, enables each token to attend to all other tokens in
the sequence.
This allows the model to capture relationships regardless of distance in the input.
Multi-head attention enables the model to jointly attend to information from different representation
subspaces.
The attention mechanism is a core part of architectures like BERT, GPT, and T5.
It supports language understanding, generation, translation, an

RAG Pipeline

In [34]:
from langchain_core.prompts import ChatPromptTemplate

# Define the RAG-style prompt
rag_prompt = """You are an assistant who is an expert in question-answering tasks.
Answer the following question using only the following pieces of retrieved context.
If the answer is not in the context, do not make up answers, just say that you don't know.
Keep the answer detailed and well formatted based on the information from the context.

Question:
{question}

Context:
{context}

Answer:
"""

# Create a ChatPromptTemplate
rag_prompt_template = ChatPromptTemplate.from_template(rag_prompt)


In [35]:
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI

# Initialize the ChatGPT model
chatgpt = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

# Function to format retrieved documents
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Construct the RAG pipeline
qa_rag_chain = (
    {
        "context": similarity_retriever | format_docs,
        "question": RunnablePassthrough()
    }
    | rag_prompt_template
    | chatgpt
)


In [36]:
qa_rag_chain.invoke("What is a Vision Transformer?")


AIMessage(content='A Vision Transformer (ViT) is a model that adapts the Transformer architecture for computer vision tasks by converting images into sequences of patches. Here are the key details about Vision Transformers:\n\n1. **Patch Processing**: In ViTs, images are divided into smaller patches, which are then flattened and embedded. This embedded representation is subsequently passed to a standard Transformer encoder.\n\n2. **Contrast with CNNs**: This method of processing images is different from Convolutional Neural Networks (CNNs), which utilize localized filters and hierarchical representations to analyze images.\n\n3. **Performance**: Vision Transformers have demonstrated the capability to outperform traditional CNNs, particularly when provided with sufficient data and computational resources.\n\n4. **Variants**: There are several variants of Vision Transformers, including DeiT (Data-efficient Image Transformers), Swin Transformer, and PiT (Pooling-based Image Transformers),

In [39]:
query = "What is LangGraph?"
result = qa_rag_chain.invoke(query)
display(Markdown(result.content))

I don't know.